<br><br><br><br><br>

# The Scientific Python Ecosystem

<br><br><br><br><br>

<br><br><br><br><br>

<p style="font-size: 1.25em; text-align: center;"><b>Who has ever used Python?</b> <i>(Show of hands.)</i></p>

<br><br><br><br><br>

<br><br><br><br><br>

<p style="font-size: 1.25em; text-align: center;"><b>Who has used Python more than C or C++?</b> <i>(Show of hands.)</i></p>

<br><br><br><br><br>

<br><br><br><br><br>

<p style="font-size: 1.25em; text-align: center;"><b>Who has ever used PyROOT?</b> <i>(Show of hands.)</i></p>

<br><br><br><br><br>

<br><br><br><br><br>

<p style="font-size: 1.25em; text-align: center;"><b>Who has ever used Numpy?</b> <i>(Show of hands.)</i></p>

<br><br><br><br><br>

<br><br><br><br><br>

<p style="font-size: 1.25em; text-align: center;"><b>Who has ever used Matplotlib?</b> <i>(Show of hands.)</i></p>

<br><br><br><br><br>

<br><br><br><br><br>

<p style="font-size: 1.25em; text-align: center;"><b>Who has ever used Pandas?</b> <i>(Show of hands.)</i></p>

<br><br><br><br><br>

<br><br><br><br><br>

<p style="font-size: 1.25em; text-align: center;"><b>Who has used Python for machine learning?</b> <i>(Show of hands.)</i></p>

<br><br><br><br><br>

<br><br><br><br><br>

### Part 1: Why Python in particle physics?

<br><br><br><br><br>

<br>

<p style="font-size: 1.25em">I could point to its broad adoption as a programming language...</p>

<center><img src="img/pypl-2019.png" width="75%"></center>

<br>

<br>

<p style="font-size: 1.25em">But it is more relevant to point to its use in data analysis.</p>

<center><img src="img/python-r-cpp-googletrends-dataset.png" width="75%"></center>

<br>

<center><img src="img/python-r-cpp-googletrends-machinelearning.png" width="75%"></center>

<br>

<br>

<p style="font-size: 1.25em">It's hard to overstate the scale of these communities.</p>

<center><img src="img/root-spark-pandas-google-trends.png" width="75%"></center>

<br>

<p style="font-size: 1.1em">There is value in adopting popular tools: every question/error message is googlable...</p>

<br>

<p style="font-size: 1.25em">The growth of Python in astronomy is... astronomical.</p>

<table width="100%"><tr style="background: white"><td><img src="img/mentions-of-programming-languages.png" width="100%"></td><td><img src="img/ligo-notebook.png" width="100%"></td></tr></table>

<br><br>

<p style="font-size: 1.25em">It's the language of choice for some—but not all—LHC experiments.</p>

<img src="img/github-cmssw-lin.png" width="100%">

<br><br>

<br><br>

<p style="font-size: 1.25em">It's the language of choice for some—but not all—LHC experiments.</p>

<img src="img/github-alice-lin.png" width="100%">

_(Can't measure ATLAS and LHCb because of private repos on GitLab.)_

<br><br>

<br>

<img src="img/commute-by-plane.png" width="60%"> _(Stolen from Jake Vanderplas.)_

<br>

<br><br>

<p style="font-size: 1.25em; font-weight: bold">Drive to the airport, then take a plane:</p>

   * Not everything needs to be fast, only the part that scales with the number of events (or other large number, like number of histogram bins or MC toys).
   
     The rest of the analysis code is bookkeeping: convenience outweighs speed.
     
   * Need to step up from interactive tinkering to full-scale analysis __*in small steps*__. Scale-up "quasistatically" to avoid a big round of bug-hunting.

<br><br>

In [ ]:
# Example: code to compute a fractal (calendar/mousepad/T-shirt...).
import time, numpy

def run_python(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.full(c.shape, maxiterations, dtype=numpy.int32)
    for h in range(height):
        for w in range(width):               # for each pixel (h, w)...
            z = c[h, w]
            for i in range(maxiterations):   # iterate at most 20 times
                z = z**2 + c[h, w]           # applying z → z² + c
                if abs(z) > 2:               # if it diverges (|z| > 2)
                    fractal[h, w] = i        # color with the iteration number
                    break                    # we're done, no need to keep iterating
    return fractal

starttime = time.time()
fractal = run_python(800, 1200)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (800 * 1200)))

In [ ]:
# Pretty, isn't it?

%matplotlib inline
import matplotlib.pyplot
fig, ax = matplotlib.pyplot.subplots(figsize=(10, 5)); ax.imshow(fractal);

In [ ]:
# REMINDER: the original code...


def run_python(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.full(c.shape, maxiterations, dtype=numpy.int32)



    for h in range(height):
        for w in range(width):               # for each pixel (h, w)...
            z = c[h, w]
            for i in range(maxiterations):   # iterate at most 20 times
                z = z**2 + c[h, w]           # applying z → z² + c
                if abs(z) > 2:               # if it diverges (|z| > 2)
                    fractal[h, w] = i        # color with the iteration number
                    break                    # we're done, no need to keep iterating
    return fractal





In [ ]:
# 50× to 100× faster...
import numba

def run_numba(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.full(c.shape, maxiterations, dtype=numpy.int32)
    return tight_loop(height, width, maxiterations, c, fractal)
@numba.jit
def tight_loop(height, width, maxiterations, c, fractal):
    for h in range(height):
        for w in range(width):               # for each pixel (h, w)...
            z = c[h, w]
            for i in range(maxiterations):   # iterate at most 20 times
                z = z**2 + c[h, w]           # applying z → z² + c
                if abs(z) > 2:               # if it diverges (|z| > 2)
                    fractal[h, w] = i        # color with the iteration number
                    break                    # we're done, no need to keep iterating
    return fractal

starttime = time.time()
fractal = run_numba(3200, 4800)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (3200 * 4800)))

<br><br>

<p style="font-size: 1.25em">The "catch" is that code in the loop must be purely numerical: arrays and basic number types. In other words, code that doesn't take advantage of "Pythonness," code that would be just as easy to write in C.</p>

<br>

<p style="font-size: 1.25em">That's what <tt>@numba.jit</tt> does: it compiles the Python function (directly to LLVM and then machine code).</p>

<br>

<p style="font-size: 1.25em">So... why not just write C code?</p>

<br><br>

<br><br><br><br>

<p style="font-size: 1.25em"> If you developed your analysis in interactive Python, in a notebook or command prompt, isolating the numerical part into a function ("<tt>tight_loop</tt>" in the previous example) is usually easier than linking to code written in another library.</p>

<br><br><br><br>

<p style="font-size: 1.25em">Fully rewriting in C++ isn't a lot faster (30% in this case), but a lot more effort.</p>

| Method                                     | time (ns/px) | speedup |
|:-------------------------------------------|-------------:|--------:|
| Pure Python                                        | 12000   |    1× |
| Vectorized Numpy                                   |   368   |   30× |
| Vectorized CuPy (run on GPU)                       |    81   |  150× |
| **Compiled by Numba**                                  |   **136**   |   **90×** |
| Compiled & parallelized by Numba                   |    45   |  250× |
| Compiled & run on GPU by Numba                     |     7.8 | 1500× |
| Parallelized by Dask                               |   238   |   50× |
| Parallelized by Dask, compiled by Numba            |    48   |  250× |
| Partially rewritten in Cython (Python/C++ hybrid)  |  1485   |    8× |
| **Completely rewritten in Cython (pure C++)**          |    **99**   |  **120×** |
| **Completely rewritten in pybind11 (pure C++)**        |    **98**   |  **120×** |
| Completely rewritten in ROOT (pure C++ with `-O0`) |   379   |   32× |

_(See [misc-fractal.ipynb](misc-fractal.ipynb) for a derivation of the above.)_

<br><br>

<p style="font-size: 1.25em"><b>Drive/bike/skateboard on your daily commute:</b> do exploration and problem-solving in Python because it has simple data structures, doesn't seg-fault, and dumps stack traces...</p>

<br>

<p style="font-size: 1.25em"><b>Fly to Europe:</b> optimize the loop that scales with big numbers so that you can finish analyzing your 100 TB this year...</p>

<br>

<p style="font-size: 1.25em; text-align: right; margin-right: 10%">... by replacing critical code <i><b>in small steps</b></i>.</p>

<br>

<center><img src="img/numpy-logo.png" width="35%"></center>

<br>

<p style="font-size: 1.25em">Numpy is the common (in-memory) data format for scientific Python.</p>

<br>

<p style="font-size: 1.25em">Nearly every package can input/output data as Numpy arrays.</p>

<br>

<p style="font-size: 1.25em">Numpy also has a suite of functions for doing calculations a whole array at a time: a <b>S</b>ingle (Python) <b>I</b>nstruction on <b>M</b>ultiple <b>D</b>ata.</p>

<br>

In [ ]:
# Numpy arrays of physics data:
import uproot
arrays = uproot.open("data/Zmumu.root")["events"].arrays(namedecode="utf-8")

print("Names of arrays in this dict:\n")
print(list(arrays), "\n\n")

arrays

In [ ]:
# Take arrays out of the dict and make each one a variable.
for n in arrays:
    exec(f"{n} = arrays['{n}']")

# Example array: energy of first muon in each event
E1

In [ ]:
# Compute pT of all muon pairs:

import numpy

pt = numpy.sqrt((px1 + px2)**2 + (py1 + py2)**2)
pt

In [ ]:
# And __LOOK__:

matplotlib.pyplot.hist(pt, bins=100, range=(0, 200));

In [ ]:
# Compute p of all muon pairs:

p = numpy.sqrt(pt**2 + (pz1 + pz2)**2)
p

In [ ]:
# And __LOOK__:

matplotlib.pyplot.hist(p, bins=100, range=(0, 500));

In [ ]:
# Compute mass of all muon pairs:

mass = numpy.sqrt((E1 + E2)**2 - p**2)
mass

In [ ]:
# And __LOOK__:

matplotlib.pyplot.hist(mass, bins=100, range=(0, 120));

In [ ]:
# Start adding cuts, exploring different regions, etc.

matplotlib.pyplot.hist(mass[Q1 != Q2], bins=100, range=(0, 120));
matplotlib.pyplot.hist(mass[Q1 == Q2], bins=100, range=(0, 120));

<br><br><br>

<p style="font-size: 1.25em">This is the point: you calculate <i>one thing</i> and then you <i>LOOK</i> at the result.</p>

<br>

<p style="font-size: 1.25em">Array-at-a-time logic gives you a statistical view of each <i>step</i> in your calculation as you develop it.</p>

<br>

<p style="font-size: 1.25em">It's not always about the speed; sometimes it's about the interactivity.</p>

<br><br><br>

In [ ]:
# As physicists, we know this. That's why we have TTree-at-a-time operations.

import ROOT
canvas = ROOT.TCanvas("canvas", "", 400, 300)
file = ROOT.TFile("data/Zmumu.root")
tree = file.Get("events")
tree.Draw("sqrt((E1 + E2)**2 - (px1 + px2)**2 - (py1 + py2)**2 - (pz1 + pz2)**2)")
canvas.Draw()

<br><br><br>

<p style="font-size: 1.25em">The hard part is turning those <tt>TTree::Draw</tt> expressions into a full analysis.</p>

<br>

<p style="font-size: 1.25em">How many of you have started with <tt>TTree::Draw</tt> and had to rewrite everything as a C++ loop?</p>

<br>

<p style="font-size: 1.25em">Our goal is to do initial exploration in a convenient way and then scale up without having to change everything.</p>

<br><br><br>

<br><br><br><br><br>

### Part 2: There's an app for that

<br><br><br><br><br>

<img src="img/shells-1.png" width="80%">

<i>(Stolen from Jake Vanderplas.)</i>

<img src="img/shells-2.png" width="80%">

<i>(Stolen from Jake Vanderplas.)</i>

<img src="img/shells-3.png" width="80%">

<i>(Stolen from Jake Vanderplas.)</i>

<img src="img/shells-4.png" width="80%">

<i>(Stolen from Jake Vanderplas.)</i>

<img src="img/shells-5.png" width="80%">

<i>(Stolen from Jake Vanderplas.)</i>

<br><br>

<p style="font-size: 1.25em">Much of what we do, as physicists, are not new problems. You can learn a lot by attempting to write an algorithm yourself, but eventually you'll want to plug together functions from established libraries (that you understand!).</p>

<br>

<p style="font-size: 1.25em"><b>Wikipedia</b> (to learn the names of things) + <b>StackOverflow</b> (to find common solutions) is a good way to develop analysis code.</p>

<br>

<p style="font-size: 1.25em">Most of these solutions come in Numpy-shaped pieces.</p>

<br><br>

<br><br>

<p style="font-size: 100px"><img src="img/scipy-logo.png" width="45%" style="vertical-align: middle"> = <img src="img/numerical-recipes.jpg" width="25%" style="vertical-align: middle"></p>

<br><br>

<br>

<p style="font-size: 1.25em">SciPy was originally the "all in one" scientific package, but today, much of the development is beyond SciPy.</p>

| Date | Development |
|:----:|:-----|
| 1994 | **Python** 1.0 released. |
| 1995 | **Numeric** was the first array package (a.k.a. Numerical, Numerical Python, Numpy). |
| 2001 | **SciPy** gathered scientific functions into one codebase, including **Numeric**. |
| 2003 | **Matplotlib** released (at that time, one of many plotters... R.I.P. **Biggles**). |
| 2003 | **Numarray** introduced as a competitor to **Numeric** with more features (memory-mapped files, alignment, record arrays). |
| 2005 | **Numpy** unified features of **Numeric** and **Numarray** and became the common array library. |
| 2008 | **Pandas** first released. |
| 2010 | **Scikit-Learn** first released. |
| 2011 | **AstroPy** first released. |
| 2012 | **Anaconda** first released. |
| 2014 | **Jupyter** first released. |
| 2015 | **Keras** first released. |

<center><img src="img/scipy-docs.png" width="80%"></center>

In [ ]:
import scipy.stats

?scipy.stats

In [ ]:
?scipy.stats.crystalball

In [ ]:
x = numpy.linspace(-10, 10, 100)
y = scipy.stats.crystalball.pdf(x, beta=0.5, m=3)     # logpdf, cdf

matplotlib.pyplot.plot(x, y);

In [ ]:
scipy.stats.crystalball.rvs(beta=0.5, m=3, size=200)

In [ ]:
masses = uproot.open("data/Zmumu.root")["events"].array("M")

y, edges = numpy.histogram(masses, bins=100)
yerr = numpy.sqrt(y)

# the middle of each bin
x = (edges[1:] + edges[:-1])/2

matplotlib.pyplot.errorbar(x, y, yerr, fmt="o", capsize=3);

In [ ]:
import scipy.optimize

def f(x, a, b, c, d):
    return a*scipy.stats.cauchy.pdf(x, b, c) + d/x**2

parameters, errors = scipy.optimize.curve_fit(f, x[y > 0], y[y > 0], sigma=yerr[y > 0])  # exclude y == 0

matplotlib.pyplot.plot(x, f(x, *parameters))
matplotlib.pyplot.errorbar(x, y, yerr, fmt="o", capsize=3);

<br><br><br><br><br>

<p style="font-size: 1.25em"><b>Five minute challenge:</b> using only commands from the previous cell, plot the fit residuals.</p>

<br><br><br><br><br>

<img src="img/iminuit.png" width="100%">

In [ ]:
import iminuit

def chi2(a, b, c, d):
    return ((y[y > 0] - f(x[y > 0], a, b, c, d))**2 / yerr[y > 0]**2).sum()

m = iminuit.Minuit(chi2, errordef=1,
                   a=3350, b=91, c=2, d=30,
                   error_a=100, error_b=1, error_c=0.1, error_d=10)
m.migrad()
m.hesse()
m.minos()
m.draw_mncontour("a", "b", nsigma=4);

<br><br><br>

<center><img src="img/scikit-learn-logo.png" width="40%"></center>

<br><br><br>

<br><br><br>

<p style="font-size: 1.25em">To branch out in a modular way, SciPy introduced the idea of "SciKits"—separate packages from SciPy that have a similar interface.</p>

<br>

<p style="font-size: 1.25em">The most famous of these is Scikit-Learn, a package that gathered all machine learning algorithms under one roof—just before the deep learning revolution...</p>


<br><br><br>

<img src="img/scikit-learn-estimators.png" width="100%">

In [ ]:
import sklearn.datasets
X1, y1 = sklearn.datasets.make_gaussian_quantiles(
    cov=2.0, n_samples=800, n_features=2, n_classes=2, random_state=1)
X2, y2 = sklearn.datasets.make_gaussian_quantiles(
    mean=(3, 3), cov=1.5, n_samples=1200, n_features=2, n_classes=2, random_state=1)
X = numpy.concatenate((X1, X2))
y = numpy.concatenate((y1, -y2 + 1))

# Example of a hard classification problem.
matplotlib.pyplot.scatter(X[y == 0, 0], X[y == 0, 1], c="deepskyblue", edgecolor="k");
matplotlib.pyplot.scatter(X[y == 1, 0], X[y == 1, 1], c="orange", edgecolor="k");

In [ ]:
import sklearn.tree

# Example model: decision tree classifier
model = sklearn.tree.DecisionTreeClassifier(max_depth=10)

# Consistent interface: nearly every model has a fit method with this signature
model.fit(X, y)

print("predictions:\n", model.predict(X))
print("truth:\n", y)

In [ ]:
import sklearn.ensemble

# Another model: boosted decision tree
model = sklearn.ensemble.AdaBoostClassifier(
    sklearn.tree.DecisionTreeClassifier(max_depth=2), algorithm="SAMME", n_estimators=100)

model.fit(X, y)

print("predictions:\n", model.predict(X))
print("truth:\n", y)

In [ ]:
xx, yy = numpy.meshgrid(numpy.arange(-5, 8, 0.02), numpy.arange(-5, 8, 0.02))
Z = model.predict(numpy.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Overlay the training points on the decision boundary.
matplotlib.pyplot.contourf(xx, yy, Z);
matplotlib.pyplot.scatter(X[y == 0, 0], X[y == 0, 1], c="deepskyblue", edgecolor="k", alpha=0.2);
matplotlib.pyplot.scatter(X[y == 1, 0], X[y == 1, 1], c="orange", edgecolor="k", alpha=0.2);
matplotlib.pyplot.xlim(-5, 8);
matplotlib.pyplot.ylim(-5, 8);

<br><br><br>

<p style="font-size: 1.25em">Notice that we have the same Numpy interfaces everywhere, and everything acts one array at a time, rather than one value at a time.</p>

<br>

<p style="font-size: 1.25em">The new deep learning frameworks define their own array types (e.g. PyTorch tensors, TensorFlow tensors), but these are very similar to Numpy arrays, with the addition that they can move data to and from GPUs.</p>

<br><br><br>

<img src="img/scikit-hep-page.png" width="100%">

<br><br><br>

<p style="font-size: 1.25em"><b>Example 1:</b> pyjet, a Numpythonic wrapper for FastJet</p>

<br><br><br>

In [ ]:
# First, make a fake-o Monte Carlo: three clusters at specified central values
fourvectors = numpy.empty(1000, dtype=[("E", float), ("px", float),
                                       ("py", float), ("pz", float)])
indexes = numpy.random.randint(0, 3, 1000)

#                        i   pt   eta    phi
for i, pt, eta, phi in [(0, 0.5,  0.5,     0),   # cluster center 0
                        (1, 0.2, -0.8,  1.57),   # cluster center 1
                        (2, 0.1,  0.3, -1.57)]:  # cluster center 2
    this = (indexes == i)
    pt = abs(numpy.random.normal(pt, 0.1, this.sum()))
    eta = numpy.random.normal(eta, 0.03, this.sum())
    phi = numpy.random.normal(phi, 0.3, this.sum())
    fourvectors["px"][this] = px = (pt*numpy.cos(phi)*numpy.cosh(eta))
    fourvectors["py"][this] = py = (pt*numpy.sin(phi)*numpy.cosh(eta))
    fourvectors["pz"][this] = pz = (pt*numpy.sinh(eta))
    fourvectors["E"][this]  = numpy.sqrt(px**2 + py**2 + pz**2)

fourvectors[:5]

In [ ]:
import pyjet

# This is FastJet, running in C++.
# 
# By passing all particles, array-at-a-time, we avoid Python's slowness.

clustering = pyjet.cluster(fourvectors, R=1.0, p=-1, ep=True)
clustering.inclusive_jets()

<br><br><br>

<p style="font-size: 1.25em"><b>Example 2:</b> particle, an interface to Particle Data Tables</p>

<br><br><br>

In [ ]:
import particle
from hepunits.units import cm

import IPython.display

# Find all strange baryons with c*tau > 1 cm
for x in particle.Particle.findall(lambda p:
    p.pdgid.is_baryon and p.pdgid.has_strange and p.width > 0 and p.ctau > 1 * cm):
    
    IPython.display.display(IPython.display.Latex("$" + x.latex_name + "$"))
    print(repr(x), end="\n\n")

<br><br><br>

<p style="font-size: 1.25em"><b>Example 3:</b> pyhf, limit-setting similar to HistFactory and CmsCombine</p>

<br><br><br>

In [ ]:
import pyhf                                                        # (scale to 125%)
import awkward

s, b, obs = [[0.0, 0.0, 0.1, 0.2, 0.5, 1.0, 1.8, 2.7, 2.0, 0.3],   # signal peaks around bin 8
             [6.0, 5.5, 4.5, 3.8, 3.3, 2.6, 2.0, 1.8, 1.5, 1.0],   # background steadily falls
             [  6,   7,   4,   4,   4,   3,   4,   3,   4,   0]]   # observations perfer 3/4 signal
model = pyhf.simplemodels.hepdata_like(signal_data=s, bkg_data=b, bkg_uncerts=numpy.sqrt(b).tolist())
def hypotest(mu):
    return pyhf.utils.hypotest(mu, obs + model.config.auxdata, model, return_expected_set=True)

mus = numpy.linspace(0, 2, 30)
CLs = awkward.fromiter([hypotest(mu) for mu in mus])
CLs_observed = CLs[:, 0, 0]                                        # mucking around with indexes
CLs_minus2, CLs_minus1, CLs_expected, CLs_plus1, CLs_plus2 = [CLs[:, 1, i, 0] for i in range(5)]

matplotlib.pyplot.fill_between(mus, CLs_minus2, CLs_plus2, facecolor="yellow");
matplotlib.pyplot.fill_between(mus, CLs_minus1, CLs_plus1, facecolor="limegreen");
matplotlib.pyplot.plot(mus, CLs_expected, c="black", linestyle="dotted");
matplotlib.pyplot.plot(mus, CLs_observed, c="black", marker="o");

<br><br><br><br><br>

### Part 3: ROOT data in Python

<br><br><br><br><br>

In [ ]:
import ROOT                                   # PyROOT

file = ROOT.TFile("data/HZZ-objects.root")    # PyROOT transliterates C++ to Python
tree = file.Get("events")

canvas = ROOT.TCanvas("canvas", "", 400, 300) # JupyROOT only: must create TCanvas
tree.Draw("muonp4.Pt()")
canvas.Draw()                                 # and Draw it to see plots inline

In [ ]:
# PyROOT can iterate over the data directly, in a Python-friendly way.

for i, event in enumerate(tree):
    print("event", i)
    for muon in event.muonp4:
        print(repr(muon), muon.Pt())
    if i > 10:
        break

<br><br>

<p style="font-size: 1.25em">But be forewarned: PyROOT was not made to be used in a loop over big data. (Actually, it's as much slower than Python as Python is from C++.)</p>

<br>

<p style="font-size: 1.25em">You can start this way, but if you'll be analyzing TB of data, you'll have to rewrite your code.</p>

<br>

<p style="font-size: 1.25em">We want to explore data in a way that doesn't have to be completely rewritten for speed.</p>

<br><br>

In [ ]:
# PyROOT's new AsMatrix method gives you Numpy arrays directly (loops run in C++).

tree.AsMatrix(["eventweight"])

In [ ]:
# But it's only for purely numeric data, not objects...

tree.AsMatrix(["MET"])

In [ ]:
# ... and not variable number of values per event, like vector<float>

tree.AsMatrix(["muoniso"])

In [ ]:
# ROOT's new preferred way of processing TTrees is called RDataFrame.
# You can define a dataflow in Python with C++ in strings (convenient!).

rdf = ROOT.RDataFrame("events", "data/HZZ-objects.root")
h = (rdf.Filter("muonp4.size() >= 2")
        .Define("zmass", r"""(muonp4[0] + muonp4[1]).M()""")
        .Histo1D(("", "", 120, 0, 120), "zmass"))
h.Draw(); canvas.Draw()

In [ ]:
# And you can get the result of this dataflow as Numpy arrays (even better!).

array = (rdf.Filter("muonp4.size() >= 2")
            .Define("zmass", r"""(muonp4[0] + muonp4[1]).M()""")
            .AsNumpy(columns=["zmass"]))
array

In [ ]:
# You can also get arrays of objects and arrays of vectors of objects.

array = rdf.AsNumpy(columns=["muonp4"])["muonp4"]
array

In [ ]:
# But looping over them is back to PyROOT. The array is an array of PyROOT objects.

for i, event in enumerate(array):
    print("event", i)
    for muon in event:
        print(repr(muon), muon.Pt())
    if i > 10:
        break

<br><br><br>

<p style="font-size: 1.25em">To get efficient processing in ROOT, you have to do the heavy work in C++. That's just how it works. What's new is that the C++ can be expressed as inline strings in Python.</p>

<br><br><br>

<br><br>

<p style="font-size: 1.25em">uproot is an alternative ROOT I/O implemented in Python + Numpy (i.e. it's pip-installable).</p>

<br>

<center><img src="img/abstraction-layers.png" width="80%"></center>

<br><br>

In [ ]:
import uproot

file = uproot.open("data/HZZ-objects.root")
tree = file["events"]

array = tree.array("muonp4")
array

In [ ]:
# Although this looks like an array of variable-length arrays of TLorentzVectors,
# it's implmented in terms of columnar arrays, not objects.
print(array.layout)
print(f"\narray for TLorentzVector.fX:\n{array.layout[2, 0, 4].array}")

In [ ]:
# So the interface is array-at-a-time: neither for loops nor call-outs to C++.

good_events = (array.counts >= 2)
print(f"good_events: {good_events}")

first  = array[good_events, 0]
second = array[good_events, 1]
print(f"\nfirst:       {first}")
print(f"second:      {second}")

z_candidates = first + second
print(f"\nz_mass:      {z_candidates.mass}")

<br><br><br><br><br>

<p style="font-size: 1.25em; text-align: center">Which one should you use?</p>

<br><br><br><br><br>

<br><br><br><br><br>

<p style="font-size: 2em; font-weight: bold; text-align: center">Both!</p>

<br><br><br><br><br>